In [4]:
import syft as sy
import pandas as pd
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split

# Initialize a couple of virtual machines (workers)
bob_vm = sy.VirtualMachine()
alice_vm = sy.VirtualMachine()

# Get clients to each VM
bob_client = bob_vm.get_root_client()
alice_client = alice_vm.get_root_client()

# Load and preprocess the dataset
file_path = 'Datasets/creditcard.csv'
data = pd.read_csv(file_path)
X = torch.tensor(data.drop('Class', axis=1).values, dtype=torch.float32)
y = torch.tensor(data['Class'].values, dtype=torch.float32)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data for the virtual clients
X_train_bob, X_train_alice = X_train[:len(X_train)//2], X_train[len(X_train)//2:]
y_train_bob, y_train_alice = y_train[:len(y_train)//2], y_train[len(y_train)//2:]

# Send the data to the workers
X_train_bob_ptr = X_train_bob.send(bob_client)
y_train_bob_ptr = y_train_bob.send(bob_client)
X_train_alice_ptr = X_train_alice.send(alice_client)
y_train_alice_ptr = y_train_alice.send(alice_client)

# Define a simple model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    # Train on Bob's data
    optimizer.zero_grad()
    prediction_bob = model(X_train_bob_ptr)
    loss_bob = criterion(prediction_bob.squeeze(), y_train_bob_ptr)
    loss_bob.backward()
    optimizer.step()

    # Train on Alice's data
    optimizer.zero_grad()
    prediction_alice = model(X_train_alice_ptr)
    loss_alice = criterion(prediction_alice.squeeze(), y_train_alice_ptr)
    loss_alice.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} - Bob's Loss: {loss_bob.get()} - Alice's Loss: {loss_alice.get()}")


0.8.3


KeyboardInterrupt: 